<a href="http://landlab.github.io"><img style="float: left" src="https://raw.githubusercontent.com/landlab/tutorials/release/landlab_header.png"></a>

## Introduction to Landlab Components

A "potluck" is a common American tradition in which each guest brings a dish to share. These contributed dishes usually come in one of a few categories: salads, drinks,  main dishes, desserts. The meal comes together as a collection of components, each of which contributes to fulfilling one of these basic roles. The resulting meal provides guests with a great variety of choices. A guest can compose their own complete and unique meal by combining their own choices for components. And each guest, while contributing their own particular dish, gets to share in the creations of their compatriots. There's a bit of standardization---the dishes are sized to fit on the tables, and their contents are accessible to the usual range of serving utensils---but with that standardization lies a great range of creativity.

Component modeling is a bit like the potluck tradition: one can construct a complete simulation by assembling components to represent the different parts of the system to be modeled. In Landlab, a **component** is a semi-standardized Python *class* that represents a particular process or calculation. Components are not stand-alone programs, but rather are designed to be used within another Python program that creates an integrated model or workflow.

<img src="./media/Eat_Alberta_Potluck.jpg">
*Figure 1: A potluck meal ("Eat Alberta Potluck" from Wikimedia Commons).*


### Examples of Landlab Components

#### Infiltration and overland flow

Here's an example from the Landlab tutorial *Green-Ampt infiltration and kinematic wave overland flow*. The example code in the tutorial uses two components, `SoilInfiltrationGreenAmpt` and `KinwaveImplicitOverlandFlow`, to calculate overland flow and infiltration across a terrain that is represented by a digital elevation model.

<img src="./media/bijou_gully_topography.png">
*Figure 2: topography of a gullied area on the Colorado high plains (from Landlab tutorial "Green-Ampt infiltration and kinematic wave overland flow").*

<img src="./media/bijou_gully_infiltration.png">
*Figure 3: cumulative infiltration water depth during a simulated five-minute heavy rainfall (from Landlab tutorial "Green-Ampt infiltration and kinematic wave overland flow")*

The following code snippets illustrate how the components are imported...

```python
from landlab.components import SoilInfiltrationGreenAmpt
from landlab.components import KinwaveImplicitOverlandFlow
```

...instantiated...

```python
infiltration = SoilInfiltrationGreenAmpt(grid)

overland_flow = KinwaveImplicitOverlandFlow(
    grid, runoff_rate=90.0, roughness=0.1, depth_exp=5.0 / 3.0
)
```

...and run in a time loop:

```python
for _ in range(nsteps):
    overland_flow.run_one_step(dt)
    infiltration.run_one_step(dt)
```

### The Landlab component collection

The current collection of components spans a variety of types of process and analysis. You can find a list [in the documentation](https://landlab.csdms.io).

To see a list, you can use the following python commands:

In [ ]:
from landlab.components import COMPONENTS

for component in COMPONENTS:
    print(component.__name__)

### Sediment diffusion with a Landlab component

Finally, we'll take a look at how Landlab components can simplify model creation even further.

Recall that in the `for` loop above, you managed to create a working sediment diffusion model with just five lines of code. However, before running the `for` loop you had to independently create the fields and parameters involved in the diffusion problem. <b>This is where Landlab's component library comes in handy!</b>

The Landlab component library is composed of individual, interoperable code packages ("components") that each represent a single Earth surface process. Examples of components include flow routing algorithms, a variety of fluvial processes, and yes, soil processes! 

In this final part of our clinic, we'll make use of Landlab's `LinearDiffuser` component to replicate our diffused fault scarp landscape. This time, we'll let the component do more of the work of field and parameter creation for us.

Start by importing the component `LinearDiffuser` from `landlab.components`.

In [ ]:
from landlab import RasterModelGrid
from landlab.components import LinearDiffuser

Great, we've imported a component which is going to help us build a diffusion model. Before we really start to use this component, though, we want to get some basic information on how it works. This information is stored in component "properties." One such property is `input_var_names`. Call any property using the syntax `ComponentName.property_name`.

In [ ]:
# check out input variable names
LinearDiffuser.input_var_names

This tells us that the only input variable required by this component is an elevation field, so we'll need to create that ourselves. 

Other standard properties are `output_var_names` and `optional_var_names`; pass an input or output name to `var_loc`, `var_type`, `var_units`, and `var_definition` to get the centering ('node', 'link', etc), array dtype (float, int), units (meters, etc) and a descriptive string, respectively. `var_help` will give you a lot of this information at once:

In [ ]:
# ask for info related to the input variable 'topographic__elevation'
LinearDiffuser.var_help("topographic__elevation")

If we want to see the complete documentation, we can also access this information using `?`.

In [ ]:
# recall the question mark notation used to access documentation in Landlab
?LinearDiffuser

Note we haven't needed to <i>instantiate</i> the component in order to access any of this information. Instantiation occurs when we create a unique instance of the component with specific variables and parameter values (just like how we created <i>instances</i> of `RasterModelGrid` above with unique names, dimensions, and grid spacing.) Don't worry if this is confusing right now - it will make sense momentarily. 

Since we know `LinearDiffuser` requires a topographic elevation field in order to be instantiated, we need to create that field. Recall that `topographic__elevation` is tied to grid nodes, so we actually need to create a grid instance, and add the `topographic__elevation` field onto the grid.

Create a new grid instance for the `LinearDiffuser` component to run on. Give you grid 25 rows and colums with a node spacing of 10.0 m.

In [ ]:
# create a new grid here

<details>
    <summary>👉 <b>click to see solution</b></summary>

```python
grid = RasterModelGrid((25, 25), xy_spacing=10)
```
</details>

Great, we now have a grid on which we can implement our diffusion component. Recall, however, our diffusion equation from above:

$$\frac{\partial z}{\partial t} = D \nabla^2 z$$

As before, we need some topographic variation in order to drive diffusion. We'll still need to add our `topographic__elevation` field manually, and we'll need to create a "fault" on the grid by elevating half of the nodes.

In [ ]:
# add a field of zeros called "topographic__elevation" and attach it to the grid nodes
# save the field to an array with a new name

<details>
    <summary>👉 <b>click to see solution</b></summary>

```python
z = grid.add_zeros("topographic__elevation", at="node")
```
</details>

In [ ]:
# now elevate the upper half of the landscape

<details>
    <summary>👉 <b>click to see solution</b></summary>

```python
z[grid.y_of_node > 120] += 10.0
```
</details>

In [ ]:
# now display the landscape

<details>
    <summary>👉 <b>click to see solution</b></summary>

```python
grid.imshow("topographic__elevation")
```
</details>

As in part [Sediment diffusion](#Sediment-diffusion), let's close the left and right grid boundaries.

In [ ]:
# close left and right boundaries

<details>
    <summary>👉 <b>click to see solution</b></summary>

```python
grid.status_at_node[grid.nodes_at_left_edge] = grid.BC_NODE_IS_CLOSED
grid.status_at_node[grid.nodes_at_right_edge] = grid.BC_NODE_IS_CLOSED
```
</details>

Now, rather than building our own diffusion model from scratch, we'll let Landlab's `LinearDiffuser` component do the work for us. Create an instance of this component named `diffusion` and pass your grid to the component.

In [ ]:
# instantiate the linear diffuser component here

<details>
    <summary>👉 <b>click to see solution</b></summary>

```python
diffusion = LinearDiffuser(grid)
```
</details>

Great, we now have an instance of our component. Next, use the `run_one_step` method with a timestep `dt` on `diffusion` in a `for` loop in order to evolve our faulted landscape. Run the loop for 25 steps. Use a same timestep, `dt`, of 1000.0. 

In [ ]:
# create your for loop here
# hint: you only need two lines of code in this cell to run the model

<details>
    <summary>👉 <b>click to see solution</b></summary>

```python
for _ in range(25):
    diffusion.run_one_step(1000.0)
```
</details>

Great, now visualize your landscape to see if it looks as you expect.

In [ ]:
# visualize landscape

<details>
    <summary>👉 <b>click to see solution</b></summary>

```python
grid.imshow("topographic__elevation")
```
</details>

Fantastic! Your final output should look very simialr to the diffusion model you created previously, but here you can see that using a Landlab component to simulate the diffusion process has simplified your `for` loop even further. You can appreciate how valuable this simplicity is, especially if you wanted to couple several components together (for example, flow routing, fluvial erosion, and hillslope diffusion) in order to evolve a more complex landscape.

As a final, fun exercise, check out the `LinearDiffuser` documentation using `?` once again. See if you can find the list of input parameters. These are listed in the order of the arguments taken by the component; the first one will always be `grid`. 

In [ ]:
# check out the optional variables

<details>
    <summary>👉 <b>click to see solution</b></summary>

```python
?LinearDiffuser
```
</details>

- What is the name of this first optional argument? It is listed right after `grid`. Optional arguments have default values, which means the user does not need to specify its value. The user can, however, change the value if desired.
- What is its default value?
- Modify your existing instance of `LinearDiffuser` to include a different value for the first optional argument. 
> Hint: go back to the cell with the comment "instantiate the linear diffuser component here," and work from there
- Re-run the `for` loop to evolve the landscape, visualize the output, and note how changing the value of this optional argument impacts the form of the evolved landscape.

Congratulations on making it to the end of this tutorial!

**Click here for more** <a href="https://landlab.csdms.io/generated/tutorials/index.html">Landlab tutorials</a>